##### Author contributions
Please fill out for each of the following parts who contributed to what:
- Conceived ideas: Everybody
- Performed math exercises: Everybody
- Performed programming exercises: Everybody
- Contributed to the overall final assignment: Everybody

Everybody did the assignment themselves and we voted afterwards for the one version to submit.

# Chapter 7
## Recurrent neural networks


    Hand-in bug-free (try "Kernel" > "Restart & Run All") and including all (textual as well as figural) output via Brightspace before the deadline (see Brightspace).

Learning goals:
1. Get familiar with recurrent hidden units
1. Implement a simple RNN (Elman network) in PyTorch
1. Implement an LSTM-based neural network in PyTorch

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import time

### Exercise 1  (1 point)

Consider a recurrent neural network with one input unit $x$, one sigmoid recurrent hidden unit $h$, and one linear output unit $y$. The values of $x$ are given for 3 time points in `x_t`. As this is a very small RNN, $W^i$, $W^h$ and $W^o$ are given as the scalar values `w_i`, `w_h` and `w_o` respectively. The hidden unit has an added bias `h_bias`. The hidden unit state is initialized with `0.0`. The only 'value-manipulating' activation function in this network is the sigmoid activation $\sigma(\cdot)$ on the hidden unit. 

1. Write down the forward pass of this network for a specific time point $t$. 
1. What is the value of the hidden state $h$ after processing the last input `x_t[2]`? 
1. What is the output `y` of the network after processing the last input `x_t[2]`? 

\begin{eqnarray*}
h_t &=& \sigma(W^i*x(t) + W^h*h(t-1) + b_h) \\ 
y_t &=&  W^o*h(t)\\
\end{eqnarray*}


For 1.2 and 1.3, you can either compute the solution by hand (show clearly how you arrived there, 3 decimal points) or write code to find the answer. 

In [ ]:
# inputs over times 0, 1, 2:
x_t = [9.0, 4.0, -2.0]

# weights and bias terms: 
w_i = 0.5
w_h = -1.0
w_o = -0.7
h_bias = -1.0
y_bias = 0.0

### Solution 1

In [ ]:
def sigmoid(A):
    """
    Computes the sigmoid activation function. Taken from Chapter04
    INPUT:
        A = [H N] activity matrix of H units for N examples
    OUTPUT
        Y = [H N] output matrix of H units for N examples
    """
    Y = 1 / (1 + np.exp(-A))
    return Y

In [ ]:
h = np.zeros((3))

h[0] = sigmoid(w_i * x_t[0] + h_bias)
h[1] = sigmoid(w_i * x_t[1] + w_h * h[0] + h_bias)
h[2] = sigmoid(w_i * x_t[2] + w_h * h[1] + h_bias)

y = w_o * h[2] + y_bias

### 1.2
print("h2: ",h[2])

### 1.3
print("y: ",y)

h2:  0.07534608655539546
y:  -0.05274226058877682


### Code introduction

We will apply two recurrent neural networks to learn a dynamic variant of the *adding problem*. First, run the next cell and inspect the output. 

There is a stream of inputs to the network, two at each time step. The first input unit will receive a series of decimal numbers in the interval $[-1,1]$. The second input unit will receive the numbers $0$, $-1$, or $1$. The target is the sum of the preceding two decimal numbers that came together with the number $1$ (called the marker, `x` in the generated output), and it should be produced whenever a marker has been seen. In the beginning until two of these markers have been seen, the output will stay 0. 


Below you will find two functions: 
1. `create_addition_data`: Generates sequential training data sets `X` and `T` for the dynamic *adding problem*, returns numpy array.
1. `MyDataset`: a custom PyTorch dataset that makes sure dimensions are as PyTorch likes them, and can return individual samples the way PyTorch wants them.

Note, the data are represented in a dictionary called `data`. To access the training, validation, and testing data, you can call `data["train"]`, `data["valid]`, and `data["test"]` respectively.

In [ ]:
def create_addition_data(n_samples=3000):
    # This is a dynamic variant of the adding problem. 
    
    # Random numbers in [-1.0,1.0]): 
    X1 = np.random.uniform(low=-1.0, high=1.0, size=(n_samples,) )   
    
    # Random markers [-1.0, 0.0, 1.0] (1.0 marks the numbers that should be added):
    X2 = np.random.choice([-1.0, 0.0, 1.0], size=(n_samples,), p=[0.25, 0.25, 0.5])
    
    # Combine
    X = np.vstack((X1, X2)).T.astype("float32")

    # Create targets
    T = np.zeros((n_samples, 1)).astype("float32")

    # Get indices of 1.0
    markers = np.nonzero(X2 == 1.0)[0]
    
    # Generate data
    mem = X1[markers[0]]
    for mi, marker in enumerate(markers[1:]):
        T[marker] = mem + X1[marker]
        mem = X1[marker]
                
    return X, T

In [ ]:
# Long as the markers x are sparse
X, T = create_addition_data(n_samples=100)

# Print some data
print("Data for the adding problem (x marks 1.0):")
for t in range(X.shape[0]):
    print("Time: {:03d} \t x: ({:+.3f} , {}) \t t: {:+.3f} ".format(
        t, X[t,0], 'x' if X[t,1] == 1.0 else ' ', T[t,0]))

Data for the adding problem (x marks 1.0):
Time: 000 	 x: (-0.189 , x) 	 t: +0.000 
Time: 001 	 x: (-0.906 , x) 	 t: -1.096 
Time: 002 	 x: (-0.863 , x) 	 t: -1.769 
Time: 003 	 x: (-0.278 ,  ) 	 t: +0.000 
Time: 004 	 x: (-0.109 , x) 	 t: -0.972 
Time: 005 	 x: (+0.656 , x) 	 t: +0.547 
Time: 006 	 x: (+0.475 ,  ) 	 t: +0.000 
Time: 007 	 x: (+0.478 ,  ) 	 t: +0.000 
Time: 008 	 x: (+0.981 , x) 	 t: +1.637 
Time: 009 	 x: (+0.947 ,  ) 	 t: +0.000 
Time: 010 	 x: (-0.349 ,  ) 	 t: +0.000 
Time: 011 	 x: (-0.414 ,  ) 	 t: +0.000 
Time: 012 	 x: (+0.955 ,  ) 	 t: +0.000 
Time: 013 	 x: (+0.877 , x) 	 t: +1.858 
Time: 014 	 x: (+0.580 ,  ) 	 t: +0.000 
Time: 015 	 x: (+0.370 , x) 	 t: +1.247 
Time: 016 	 x: (+0.192 , x) 	 t: +0.563 
Time: 017 	 x: (-0.501 ,  ) 	 t: +0.000 
Time: 018 	 x: (-0.074 ,  ) 	 t: +0.000 
Time: 019 	 x: (-0.948 , x) 	 t: -0.756 
Time: 020 	 x: (-0.791 ,  ) 	 t: +0.000 
Time: 021 	 x: (+0.261 , x) 	 t: -0.687 
Time: 022 	 x: (-0.157 , x) 	 t: +0.105 
Time: 023 	 x:

In [ ]:
# Make PyTorch dataset
class MyDataset(torch.utils.data.Dataset):
    
    def __init__(self, X, T):
        self.X = torch.from_numpy(X).type(torch.FloatTensor) # [n_examples, n_samples, n_features]
        self.T = torch.from_numpy(T).type(torch.FloatTensor) # [n_examples, n_samples]
        
    def __getitem__(self, index):
        return self.X[index, :, :], self.T[index]
    
    def __len__(self):
        return self.X.size()[0]

In [ ]:
n_examples = 9
n_samples = 3000
data = {}

# Define training data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["train"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=3)

# Define validation data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["valid"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=3)

# Define test data
X = np.zeros((n_examples, n_samples, 2))
T = np.zeros((n_examples, n_samples, 1))
for i_example in range(n_examples):
    X[i_example, :, :], T[i_example, :] = create_addition_data(n_samples)
data["test"] = torch.utils.data.DataLoader(MyDataset(X, T), batch_size=1)

### Exercise 2: training a network  (0.5 points)

We neede a function to train a `model`. This function `train_model(model, data, optimizer, criterion, n_epochs)` should do the following: 

1. Loop `n_epochs` times over the dataset, and loop over minibatches
1. Train the model on the training data and save the loss per epoch
1. Validate the model on the validation data and save the loss per epoch
1. The function should return the trained model and the losses

Note: this function is quite similar again as the function your wrote wor the MLP and CNN. The only difference is that we do not need to compute an accuracy, as we are performing regression here.

### Solution 2

In [ ]:
def train_model(model, data, optimizer, criterion, n_epochs):
  train_loss = np.zeros((n_epochs), dtype=np.float32)
  valid_loss = np.zeros((n_epochs), dtype=np.float32)
  start = 0

  for epoch in range(n_epochs):
    train_total = 0
    val_total = 0
    start = time.time()
    try:
      torch.nn.init.zeros_(model.hr.weight)
    except:
      pass


    #validation loop
    with torch.no_grad():
      for val_idx, minibatch in enumerate(data["valid"], 0):
        
        val_images, val_labels = minibatch[0].to(device), minibatch[1].to(device)
        val_outputs = model(val_images)
        val_loss = criterion(val_outputs, val_labels)
        valid_loss[epoch]+=val_loss.item()

    #training loop
    with torch.enable_grad():
      for train_idx, minibatch in enumerate(data["train"], 0):
        model.zero_grad()
        images, labels = minibatch[0].to(device), minibatch[1].to(device)
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        train_loss[epoch]+=loss.item()
      
    train_loss[epoch] = train_loss[epoch] / train_idx
    valid_loss[epoch] = valid_loss[epoch] / val_idx

    print("Epoch: ", epoch, " t_loss: ", train_loss[epoch], " v_loss: ", valid_loss[epoch], " t: ", time.time()-start)
    start = 0

  return model, train_loss, valid_loss

### Exercise 3: Testing a network  (1.5 points)

We neede a function to test a trained `model`. This function `test_model(model, data)` should do the following: 

1. Let `model` predict outputs on the testing data. For this, iterate through test data `data["test"]` and pass each sample through `model`. 
1. Save the model output as well as the target output
1. The function should return the predicted and target outputs

### Solution 3

In [ ]:
def test_model(model, data):
  predicted = []
  targets = []
  with torch.no_grad():
    for test_idx, minibatch in enumerate(data["test"], 0):
      test_image, test_label = minibatch[0].to(device), minibatch[1].to(device)
      test_outputs = model(test_image)
      predicted.append(test_outputs)
      targets.append(test_label)
  return predicted, targets

### Exercise 4: Simple RNN  (3 points)

We first implement a simple recurrent architecture (a simple [Elman network](http://mnemstudio.org/neural-networks-elman.htm)). 

1. First implement the linear layers `l1` and `l2`. They should lead from `n_input` input units over `n_hidden` hidden units to `n_out` output units.
1. Add a recurrent linear weight layer `hr`. These are weights that self-connect to the hidden units. The input will be the values of the `n_hidden` hidden units, and they should project back to the `n_hidden` hidden units. 
1. A forward pass will update the hidden state with the inputs and the recurrent layer weights, and produce the output from the hidden unit. Specifically you should do the following: 
    2. If we are at the first time point, the hidden state should be set to the input passed through `l1` and `tanh` activations.
    2. If the hidden state has information from previous time points: a) Pass the input through `l1`. b) Pass the hidden state through the recurrent weight layer `hr`. c) The sum of a) and b) should be passed through the `tanh` activation. d) The result should be the new hidden state (used for the next time point).
    2. Finally pass the hidden state through layer `l2`. This produces the output `y` for that time point.
1. The forward pass will receive data `x` with shape [batch_size, time_points, features]. So within the forward pass, you will have to loop over time points, performing the steps as descibed above. The output of the forward pass is then output `y` with shape [batch_size, time_points, 1].

Note: this exercise could also be done with nn.RNN(). However, we want you to understand what a RNN is doing, so we want you to use nn.Linear instead.

### Solution 4

In [ ]:
from collections import OrderedDict

class Elman(nn.Module):
    def __init__(self, n_input, n_hidden, n_out):
        super(Elman, self).__init__()
        self.l1 = nn.Linear(n_input, n_hidden)
        self.tanh = nn.Tanh()
        self.hr = nn.Linear(n_hidden, n_hidden)
        self.l2 = nn.Linear(n_hidden,n_out)

    def forward(self, x):
      y = torch.empty((x.shape[0], x.shape[1], 1), requires_grad=False).to(device)

      for time in range(x.shape[1]):
        if time == 0:
          hidden_states = self.tanh(self.l1(x[:,time]))
        else:
          a = self.l1(x[:,time])
          b = self.hr(hidden_states)
          hidden_states = self.tanh(a+b)
        y[:, time] = self.l2(hidden_states)
      return y

### Exercise 5: Setup and run (1 point)

Try your simple `RNN` with the dynamic addition task. 

1. Define the model. `RNN` should have **2 hidden units**.
1. Define the loss as the Mean Squared Error loss, and use an Adam optimizer.
1. Train your model for several epochs on the data with `train_model`.
1. Plot the train and validation losses. 
1. Test the trained model with `test_model` 
1. Plot at least one target time series together with the predicted time series

Based on the losses and predictions, what would your conclusion be? Did the simple RNN learn the task? 

### Solution 5

In [ ]:
elman = Elman(2,2,1)
torch.nn.init.zeros_(elman.l1.weight)
torch.nn.init.zeros_(elman.l2.weight)

criterion = nn.MSELoss(reduction='sum',)
optimizer = optim.Adam(elman.parameters(), lr=0.005, amsgrad=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
elman.to(device)

model, train_loss, val_loss = train_model(elman, data, optimizer, criterion, n_epochs=1000)

# Plot losses
plt.plot(range(1,train_loss.shape[0]+1),train_loss)
plt.plot(range(1,val_loss.shape[0]+1),val_loss)
plt.legend(('train_loss', 'val_loss'), loc='upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss network')
plt.show()

predicted, targets = test_model(model, data)

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[1][0])
plt.plot(range(0,3000),predicted[1][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[2][0])
plt.plot(range(0,3000),predicted[2][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[8][0])
plt.plot(range(0,3000),predicted[8][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

Epoch:  0  t_loss:  6233.7686  v_loss:  6294.411  t:  4.249569416046143
Epoch:  1  t_loss:  5963.0186  v_loss:  6024.122  t:  4.0452611446380615
Epoch:  2  t_loss:  5702.171  v_loss:  5762.5464  t:  4.038117408752441
Epoch:  3  t_loss:  5455.1963  v_loss:  5513.2905  t:  3.8663175106048584
Epoch:  4  t_loss:  5227.0137  v_loss:  5281.046  t:  3.914708137512207
Epoch:  5  t_loss:  5022.594  v_loss:  5070.735  t:  3.9353415966033936
Epoch:  6  t_loss:  4846.313  v_loss:  4886.931  t:  3.9162347316741943
Epoch:  7  t_loss:  4701.1553  v_loss:  4733.126  t:  3.951385021209717
Epoch:  8  t_loss:  4587.872  v_loss:  4610.8804  t:  3.909842014312744
Epoch:  9  t_loss:  4504.5234  v_loss:  4519.1836  t:  3.9241716861724854
Epoch:  10  t_loss:  4446.7236  v_loss:  4454.493  t:  3.9942243099212646
Epoch:  11  t_loss:  4408.2764  v_loss:  4411.2812  t:  3.91511869430542
Epoch:  12  t_loss:  4381.9688  v_loss:  4382.68  t:  3.949875831604004
Epoch:  13  t_loss:  4360.709  v_loss:  4361.4277  t:  4

### Exercise 6: LSTM RNN (2 points)

Long-Short Term Memory (LSTM) units have more [powerful functionality](http://colah.github.io/posts/2015-08-Understanding-LSTMs/), such as sele

1. `lstm` should be an `LSTM` layer leading from the `n_input` inputs to the `n_hidden` hidden units.
1. `fc` should be a fully-connected (linear) layer leading from the hidden units (output of `lstm`) to the `n_out` output units. 
1. The network does not make use of further activation functions. 

### Solution 6

In [ ]:
class Lstm(nn.Module):
  def __init__(self, n_input, n_hidden, n_out):
    super(Lstm, self).__init__()
    self.fc      = nn.Linear(n_hidden,n_out)
    self.lstm    = nn.LSTM(input_size = n_input, hidden_size=n_hidden, batch_first=True)

  def forward(self,x):
    x = self.lstm(x)
    x = self.fc(x[0])
    return x

### Exercise 7: Setup and run (1 point)

Try your `LSTM` model with the dynamic addition task. 

1. Define the model. `LSTM` should have **2 hidden units**.
1. Define the loss as the Mean Squared Error loss, and use an Adam optimizer.
1. Train your model for several epochs on the data with `train_model`.
1. Plot the train and validation losses. 
1. Test the trained model with `test_model` 
1. Plot at least one target time series together with the predicted time series

Did the LSTM network capture the task better? Did any of the two capture the task perfectly? Or are the two networks on par? 

### Solution 7

In [ ]:
lstm = Lstm(2,2,1)

criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(lstm.parameters(),lr=0.006, amsgrad=False)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device = "cpu"
elman.to(device)

model, train_loss, val_loss = train_model(lstm, data, optimizer, criterion, n_epochs=1000)

# Plot losses
plt.plot(range(1,train_loss.shape[0]+1),train_loss)
plt.plot(range(1,val_loss.shape[0]+1),val_loss)
plt.legend(('train_loss', 'val_loss'), loc='upper right')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.title('Loss network')
plt.show()

In [ ]:
predicted, targets = test_model(model, data)

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[1][0])
plt.plot(range(0,3000),predicted[1][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[2][0])
plt.plot(range(0,3000),predicted[2][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

plt.figure(figsize=([20,10]))
plt.plot(range(0,3000),targets[8][0])
plt.plot(range(0,3000),predicted[8][0])
plt.legend(('target', 'prediction'), loc='upper right')
plt.ylabel('Value')
plt.xlabel('Time')
plt.title('Time Series')
plt.show()

Did the LSTM network capture the task better? Did any of the two capture the task perfectly? Or are the two networks on par? 


1.   The LSTM network did capture the task better. We can see this in the graphs we create and we can see it in the loss.
2.   None of the two networks captured the task perfectly. In essence both networks tried to approximate the actual function as closely as possible. However it seems weird that the result is something like 2+2=3.99 which seems unintuitive.
3.  The two networks are not on par in the sense that the predictions of the lstm network were much closer to the actual values when compared to the elman network.
